In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

In [ ]:
%run "/Workspace/Users/cronies-02gorilla@icloud.com/access"

key,value
spark.databricks.delta.formatCheck.enabled,false


In [ ]:
stream_df = spark \
.readStream \
.format('kinesis') \
.option('streamName','streaming-0affea73130b-user') \
.option('initialPosition','latest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

In [ ]:
# Define the schema to speed up processing
jsonSchema = StructType([ 
                         StructField("ind", StringType(), True), 
                         StructField("age", StringType(), True), 
                         StructField("date_joined", StringType(), True), 
                         StructField("first_name", StringType(), True), 
                         StructField("last_name", StringType(), True), 
                         ])

In [ ]:
string_df = stream_df.selectExpr("CAST(data as STRING)")

In [ ]:
json_df = string_df.select(from_json(col("data"), jsonSchema))

In [ ]:
geo_stream_df = json_df.withColumn("ind", col("from_json(data)")["ind"])\
.withColumn("age", col("from_json(data)")["age"])\
.withColumn("date_joined", col("from_json(data)")["date_joined"])\
.withColumn("first_name", col("from_json(data)")["first_name"])\
.withColumn("last_name", col("from_json(data)")["last_name"])

In [ ]:
# drop from_json(data) column
cleaned_df = geo_stream_df.drop("from_json(data)")

# drop duplicates
cleaned_df = cleaned_df.dropDuplicates()

# Create a new column user_name that concatenates the information found in the first_name and last_name columns
# Drop first_name and last_name columns from the DataFrame
cleaned_df = cleaned_df.withColumn("user_name", concat("first_name", lit(" "), "last_name"))
cleaned_df = cleaned_df.drop("first_name", "last_name")

# cast int datatype to ind
cleaned_df = cleaned_df.withColumn("ind", cleaned_df["ind"].cast("int"))

# Convert the date_joined column from a string to a timestamp data type
cleaned_df = cleaned_df.withColumn("date_joined", to_timestamp("date_joined"))

# drop duplicates
cleaned_df = cleaned_df.dropDuplicates()

# Reorder the DataFrame columns to have the following column order:
cleaned_df = cleaned_df.select("ind", "user_name", "age", "date_joined")

In [ ]:
display(cleaned_df)

ind,user_name,age,date_joined
8808,Albert Adams,21,2015-11-05T22:07:56.000+0000
3683,David Anderson,21,2016-02-08T10:08:32.000+0000
414,Aaron Bartlett,21,2015-11-24T02:15:36.000+0000
4224,Alison Cole,21,2015-11-05T13:39:57.000+0000
443,Carla Atkins,49,2016-05-08T20:29:33.000+0000
5083,Sharon Woodward,44,2017-04-25T23:49:22.000+0000
9020,Natalie Davis,22,2016-02-06T07:52:27.000+0000
3937,Caroline Carlson,31,2015-10-22T08:40:35.000+0000
9582,Christopher Hawkins,45,2016-09-15T06:02:53.000+0000
2861,Elizabeth Terry,28,2016-01-29T20:32:05.000+0000


In [ ]:
output_user_df = cleaned_df

In [ ]:
output_user_df.writeStream \
  .format("delta") \
  .queryName("0affea73130b_user_query") \
  .outputMode("append") \
  .option("checkpointLocation", "/tmp/kinesis/_checkpoints/0affea73130b_user_query") \
  .table("0affea73130b_user_table")

Out[51]: <pyspark.sql.streaming.StreamingQuery at 0x7fa9beb9d520>

In [ ]:
# dbutils.fs.rm("/tmp/kinesis/_checkpoints/0affea73130b_user_query", True)